In [4]:
import requests
import json
from typing import List, Dict

class YandexSpeller:
    def __init__(self):
        self.url = "https://speller.yandex.net/services/spellservice.json/checkTexts"

    def check_text(self, text: str) -> List[List[Dict]]:
        """Проверяет текст на орфографические ошибки"""
        params = {
            'text': text,
            'lang': 'ru,en',
            'options': 14
        }

        try:
            response = requests.get(self.url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при запросе к Yandex Speller: {e}")
            return []

    def correct_text(self, text: str) -> str:
        """Исправляет ошибки в тексте"""
        sentences = text.split('. ')
        corrected_sentences = []

        for sentence in sentences:

            errors = self.check_text(sentence)

            if errors and errors[0]:
                corrected_sentence = sentence
                for error in sorted(errors[0], key=lambda x: x['pos'], reverse=True):
                    if error['s']:

                        start = error['pos']
                        end = start + error['len']
                        corrected_sentence = (corrected_sentence[:start] + error['s'][0] + corrected_sentence[end:])

                corrected_sentences.append(corrected_sentence)
            else:
                corrected_sentences.append(sentence)

        return '. '.join(corrected_sentences)


def main():
    speller = YandexSpeller()

    text_with_errors = """
Пременение указаных сакций асуществляется надлежащими образом в установленном законам порядки и подлежит
исполнению всеми сотрудниками организации незамедлительно.

Подписано руководителем подразделения. Подлежит обязательному ознакомлению всех сотрудников пад роспись.
    """

    print("Исходный текст с ошибками:")
    print(text_with_errors)
    print("\n" + "="*50 + "\n")

    corrected_text = speller.correct_text(text_with_errors)

    print("Исправленный текст:")
    print(corrected_text)

if __name__ == "__main__":
    main()

Исходный текст с ошибками:

Пременение указаных сакций асуществляется надлежащими образом в установленном законам порядки и подлежит
исполнению всеми сотрудниками организации незамедлительно.

Подписано руководителем подразделения. Подлежит обязательному ознакомлению всех сотрудников пад роспись.
    


Исправленный текст:

Применение указанных санкций осуществляется надлежащим образом в установленном законом порядке и подлежит
исполнению всеми сотрудниками организации незамедлительно.

Подписано руководителем подразделения. Подлежит обязательному ознакомлению всех сотрудников под роспись.
    
